In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Definition of a Curve
class Curve:
    # Constructor
    def __init__(self, list_of_points, closed=False):
        self.list_of_points = list_of_points
        self.closed = closed
        self.J = len(list_of_points)
    
    # Square Bracket Overload
    def __getitem__(self, key):
        return self.list_of_points[key % self.J]
    def __setitem__(self, key, value):
        self.list_of_points[key] = value
    
    # Length
    def __len__(self):
        return self.J
    
    # Curve Addition
    def __add__(self, otherCurve):
        if len(self) == len(otherCurve) and self.closed == otherCurve.closed:
            return Curve([self[i] + otherCurve[i] for i in range(len(self))])
    
    # Curve Length
    def curveLength(self):
        l = 0
        for i in range(self.J - 1):
            edgeLength = np.linalg.norm(self[i + 1] - self[i])
            l += edgeLength ** 2
        if self.closed:
            edgeLength = np.linalg.norm(self[-1] - self[0])
            l += edgeLength ** 2
        return l

In [3]:
# p, q, T are all array
def kernelalphabeta(p, q, T, alpha=2, beta=4):
    pmq = p - q
    numerator = np.linalg.norm(np.cross(T, pmq)) ** alpha
    denominator = np.linalg.norm(pmq, 1) ** beta
    return numerator / denominator

def kij(curve, i, j, alpha=2, beta=4):
    TI = curve[i+1] - curve[i]
    TI = TI / np.linalg(TI)
    res = kernelalphabeta(curve[i], curve[j], TI, alpha, beta)
    res += kernelalphabeta(curve[i], curve[j+1], TI, alpha, beta)
    res += kernelalphabeta(curve[i+1], curve[j], TI, alpha, beta)
    res += kernelalphabeta(curve[i+1], curve[j+1], TI, alpha, beta)
    return res / 4


# Pass a list of points.
def energy(points):
    J = len(points)

    e = 0
    for i in range(J):
        for j in range(J):
            if abs(i - j) > 1 and abs(i - j + J) > 1 and abs(i - j - J) > 1:
                xi = points[i % J]
                xipm = points[(i+1) % J]
                xj = points[j % J]
                xjpm = points[(j+1) % J]
                xI = xipm - xi
                lI = np.linalg.norm(xI)
                TI = xI / lI
                kernelAB = 0
                kernelAB += kernelalphabeta(xi, xj, TI) 
                kernelAB += kernelalphabeta(xi, xjpm, TI) 
                kernelAB += kernelalphabeta(xipm, xj, TI) 
                kernelAB += kernelalphabeta(xipm, xjpm, TI)
                kernelAB *= 0.25 * lI * np.linalg.norm(xjpm-xj)
                e += kernelAB
                print(f"{i}, {j}: kernelAB: {kernelAB}")

    return e


In [4]:
# Generates the index pairs that are responsible for the derivative.
# Note that 4(J-1) pairs are generated.
def derivative_index(k, J):
    index_list = []
    for i in range(J):
        if abs(k - i) > 1 and abs(k - i + J) > 1 and abs(k - i - J):
            index_list.append((k, i))
    for i in range(J):
        if abs(k - 1 - i) > 1 and abs(k - 1 - i + J) > 1 and abs(k - 1 - i - J):
            index_list.append((k-1, i))
    for i in range(J):
        if abs(k - i) > 1 and abs(k - i + J) > 1 and abs(k - i - J):
            index_list.append((i, k))
    for i in range(J):
        if abs(k - 1 - i) > 1 and abs(k - 1 - i + J) > 1 and abs(k - 1 - i - J):
            index_list.append((i, k-1))
    

    return index_list

In [7]:
def kjk(curve, p, q, r, alpha=3, beta=6):
    k = p
    j = q
    xkEdge = curve[k+1] - curve[k]
    xkEdgeLen = np.linalg.norm(xkEdge)
    xkj = curve[k] - curve[j]
    xkjLen = np.linalg.norm(xkj)
    xi = xkEdgeLen**2 * xkjLen**2 - (np.dot(xkEdge, xkj))**2
    eta = xkjLen**beta * xkEdgeLen**alpha
    dxi = -2 * xkEdge * xkjLen**2 + 2 * xkEdgeLen**2 * xkj - 2 * np.dot(xkEdge, xkj) * (xkEdge - xkj)
    deta = beta * xkjLen**(beta - 2) * xkEdgeLen**alpha * xkj + alpha * xkjLen**beta * xkEdgeLen**(alpha-2)*(-xkEdge)
    return (xi, eta, dxi, deta)

def ijk(curve, p, q, r, alpha=3, beta=6):
    i = p
    j = q
    k = r
    xkEdge = curve[k+1] - curve[k]
    xkEdgeLen = np.linalg.norm(xkEdge)
    xij = curve[i] - curve[j]
    xijLen = np.linalg.norm(xij)
    xi = xkEdgeLen**2 * xijLen**2 - (np.dot(xkEdge, xij))**2
    eta = xijLen**beta * xkEdgeLen**alpha
    dxi = -2 * xijLen**2 * xkEdge + 2 * np.dot(xkEdge, xij) * xij
    deta = alpha * xijLen**beta * xkEdgeLen**(alpha-2) * (-xkEdge)
    return (xi, eta, dxi, deta)

def km1jkm1(curve, p, q, r, alpha=3, beta=6):
    k = p + 1
    j = q
    xkEdge = curve[k] - curve[k-1]
    xkEdgeLen = np.linalg.norm(xkEdge)
    xkmj = curve[k-1] - curve[j]
    xkmjLen = np.linalg.norm(xkmj)
    xi = xkEdgeLen**2 * xkmjLen**2 - (np.dot(xkEdge, xkmj))**2
    eta = xkmjLen**beta * xkEdgeLen**alpha
    dxi = 2 * xkmjLen**2 * xkEdge - 2 * np.dot(xkEdge, xkmj) * xkmj
    deta = alpha * xkmjLen**beta * xkEdgeLen**(alpha-2) * xkEdge
    return (xi, eta, dxi, deta)

def kjkm1(curve, p, q, r, alpha=3, beta=6):
    k = p
    j = q
    xkEdge = curve[k] - curve[k-1]
    xkEdgeLen = np.linalg.norm(xkEdge)
    xkj = curve[k] - curve[j]
    xkjLen = np.linalg.norm(xkj)
    xi = xkEdgeLen**2 * xkjLen**2 - (np.dot(xkEdge, xkj))**2
    eta = xkjLen**beta * xkEdgeLen**alpha
    dxi = 2 * xkjLen**2 * xkEdge + 2 * xkEdgeLen**2 * xkj - 2 * np.dot(xkEdge, xkj) * (xkEdge + xkj)
    deta = beta * xkEdgeLen**alpha * xkjLen**(beta-2) * xkj + alpha * xkjLen**beta * xkEdgeLen**(alpha-2) * xkEdge
    return (xi, eta, dxi, deta)

def ikj(curve, p, q, r, alpha=3, beta=6):
    i = p
    j = r
    k = q
    xjEdge = curve[j+1] - curve[j]
    xjEdgeLen = np.linalg.norm(xjEdge)
    xki = curve[k] - curve[i]
    xkiLen = np.linalg.norm(xki)
    xi = xjEdgeLen**2 * xkiLen**2 - (np.dot(xjEdge, xki))**2
    eta = xkiLen**beta * xjEdgeLen**alpha
    dxi = 2 * xjEdgeLen**2 * xki - 2 * np.dot(xjEdge, xki) * xjEdge
    deta = beta * xjEdgeLen**alpha * xkiLen**(beta-2) * xki
    return (xi, eta, dxi, deta)



    

def summandComputation(curve, p, q, r, k, alpha=3, beta=6):
    # (k, j, k)
    if (p, r) == (k, k):
        print("kjk")
    # (i, j, k)
    elif r == k:
        print("ijk")
    elif (p, r) == (k-1, k-1):
        print("k-1,j,k-1")
    elif (p, r) == (k, k-1):
        print("k, j, k-1")
    elif q == k:
        print("ikj")
    else:
        print(f"NOT DEFINED: {p}, {q}, {r}")

In [18]:
dIndex = derivative_index(1, 10)
for index in dIndex:
    p, q = index
    p += 1
    q += 1
    r = p
    summandComputation(curve, p, q, r, 1)

NOT DEFINED: 2, 4, 2
NOT DEFINED: 2, 5, 2
NOT DEFINED: 2, 6, 2
NOT DEFINED: 2, 7, 2
NOT DEFINED: 2, 8, 2
NOT DEFINED: 2, 9, 2
NOT DEFINED: 2, 10, 2
kjk
kjk
kjk
kjk
kjk
kjk
kjk
NOT DEFINED: 4, 2, 4
NOT DEFINED: 5, 2, 5
NOT DEFINED: 6, 2, 6
NOT DEFINED: 7, 2, 7
NOT DEFINED: 8, 2, 8
NOT DEFINED: 9, 2, 9
NOT DEFINED: 10, 2, 10
ikj
ikj
ikj
ikj
ikj
ikj
ikj


In [6]:
points = []
resolution = 10
for i in range(resolution):
    theta = 2 * np.pi / resolution * i
    points.append(np.array([np.cos(theta), np.sin(theta), 0], dtype="float64"))
curve = Curve(points)
